## Importa funções para o pré-processamento

In [12]:
from pre_processamento import *

from dlisio import dlis
import pandas as pd

# A função 'glob' do módulo 'glob' é usada para procurar todos os arquivos em um diretório com determinada extensão
import glob

## Carrega os dados

In [13]:
dli_dict = {}        # Conterá os arquivos lógicos
filenames = []  # Conterá os caminhos dos arquivos dlis
nomes_anp = []      # Conterá os nomes ANP dos poços
charId_inicio, charId_fim = 7, 10   # Índices de início e fim da substring identificadora do poço

# Utilizando a função 'glob' do módulo 'glob' para procurar todos os arquivos com extensão DLIS em 'Data'
for file in glob.glob(r'**/Data' + "/*.dlis", recursive=True):
    
    # Carregando os arquivos dlis com 'load' e armazenando-os em 'leitura'
    # 'tail' recebe valores restantes, caso a função retorne mais de uma peça de informação
    leitura, *tail = dlis.load(f'{file}')
    
    nome = leitura.origins[0].well_name

    nomes_anp.append(nome)    # armazenando o nome do poço
    
    filenames.append(file)      # armazenando o caminho dos arquivos dlis
    
    nome_abreviado = nome[charId_inicio : charId_fim]   # identificador do poço, e.g. '900'
    
    # Armazenando o arquivo lógico como valor em um dicionário onde a chave é o identificador do poço
    dli_dict[nome_abreviado] = leitura   

## Separa TODAS as curvas presentes nos .dlis de cada poço

In [14]:
channels_dict = {}

for key, poco in dli_dict.items():
    channels = poco.frames[0].channels
    channels_list = [channel.name for channel in channels]
    channels_dict[key] = channels_list

## Salva as curvas presentes nos .dlis em arquivos CSV

In [15]:
import csv

for key, value in channels_dict.items():
    file_name = f"Curvas_CSV/curvas_{key}.csv"
    with open(file_name, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([f"Curvas do Poco {key}"])
        for item in value:
            writer.writerow([item])

    print(f"Arquivo {file_name} criado com sucesso")

Arquivo Curvas_CSV/curvas_595.csv criado com sucesso
Arquivo Curvas_CSV/curvas_900.csv criado com sucesso
Arquivo Curvas_CSV/curvas_889.csv criado com sucesso
Arquivo Curvas_CSV/curvas_912.csv criado com sucesso
Arquivo Curvas_CSV/curvas_897.csv criado com sucesso
Arquivo Curvas_CSV/curvas_910.csv criado com sucesso
Arquivo Curvas_CSV/curvas_907.csv criado com sucesso


## Concatena os arquivos CSV das curvas em um único arquivo

In [16]:
import os

pasta = "Curvas_CSV"

# Lista para armazenar os dataframes repectivos aos .csv
df_list = []

# Itera sobre a pasta Curvas_CSV
for arquivo in os.listdir(pasta):
    file_path = os.path.join(pasta, arquivo)

    # Lê o arquivo CSV como um DF
    df = pd.read_csv(file_path)
    df_list.append(df)

# Concatena os DFs ao longo do eixo das colunas
df_concat = pd.concat(df_list, axis=1)

# Salva o DF concatenado em um .csv
df_concat.to_csv("Curvas_CSV/curvas_pocos.csv", index=False)

print("Arquivo Curvas_CSV/curvas_pocos.csv criado com sucesso")

Arquivo Curvas_CSV/curvas_pocos.csv criado com sucesso


## Cria dataframes para os poços

In [17]:
dlis_df_dict = {}   # Conterá os dataframes respectivos aos poços

# Curvas de perfis escolhidas
curvas_escolhidas = ['TDEP', 'GR', 'NPHI', 'RHOB', 'DRHO', 'HDRS', 'LLD', 'BSZ', 'BS', 'CALI', 'DCALI', 'PE', 'DTC']

# Iterando sobre os arquivos lógicos de todos os poços (que estão armazenados em 'dli_dict'),
# '.values()' se refere aos valores do dicionário (não às chaves)
for chave, poco in dli_dict.items():
    try:
        # Armazenando as curvas que serão utilizadas em uma lista
        curvas_utilizadas = [
            channel.name                                    # Os elementos da lista serão os nomes das curvas
            for channel in poco.channels                    # As curvas são acessadas por meio de 'poco.channels'
            if channel.name in curvas_escolhidas            # As curvas que não utilizaremos não serão armazenadas na lista
        ]


        curvas = poco.frames[0].curves()

        # Criando um pandas dataframe do poço respectivo à atual iteração e armazenando o mesmo em dlis_df
        dlis_df_dict[chave] = pd.DataFrame(curvas[curvas_utilizadas])
    except:
        pass

## Transforma os valores -999.25 em nulos

In [18]:
for poco in dlis_df_dict.values():
    poco.replace([-999.25], [None], inplace = True)

## Aplicando os mnemônicos

In [19]:
aplica_mnemonico(dlis_df_dict, ['BS', 'BSZ'], 'BS')
aplica_mnemonico(dlis_df_dict, ['LLD', 'HDRS'], 'RESD')
aplica_mnemonico(dlis_df_dict, ['RHOB', 'RHOZ'], 'RHOB')

## Renomeia CALI para CAL

In [20]:
for poco in dlis_df_dict.values():
    renomeia_coluna(poco, 'CALI', 'CAL')

## Adiciona coluna DCAL

In [21]:
add_DCAL(dlis_df_dict)

## Temos as seguintes curvas 

In [22]:
for key, poco in dlis_df_dict.items():
    curvas = sorted(poco.keys())
    print(f"{key}: {curvas}")

595: ['BS', 'CAL', 'DCAL', 'DRHO', 'GR', 'NPHI', 'RESD', 'RHOB', 'TDEP']
900: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
889: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
912: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
897: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
910: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
907: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']


## Remove valores DRHO e DCAL indesejados

In [23]:
limita_curva(dlis_df_dict, "DRHO", -0.15, 0.15)
limita_curva(dlis_df_dict, "DCAL", -2, 2)

c:\Users\rafae\Documents\College\PIBIC\Rafael-AnaliseDados\pre_processamento.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poco[curva][poco[curva] < limite_inferior] = None
c:\Users\rafae\Documents\College\PIBIC\Rafael-AnaliseDados\pre_processamento.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poco[curva][poco[curva] < limite_inferior] = None
c:\Users\rafae\Documents\College\PIBIC\Rafael-AnaliseDados\pre_processamento.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

## Inverte a ordem das linhas dos dataframes

In [24]:
for key in dlis_df_dict.keys():
    dlis_df_dict[key] = dlis_df_dict[key].iloc[::-1]

## Salva os dados dos dataframes em arquivos CSV

In [25]:
for key, value in dlis_df_dict.items():
    file_name = f"Data_CSV/poco_{key}.csv"
    value.to_csv(file_name, index=False)
    print(f"Arquivo {file_name} criado com sucesso.")

Arquivo Data_CSV/poco_595.csv criado com sucesso.
Arquivo Data_CSV/poco_900.csv criado com sucesso.
Arquivo Data_CSV/poco_889.csv criado com sucesso.
Arquivo Data_CSV/poco_912.csv criado com sucesso.
Arquivo Data_CSV/poco_897.csv criado com sucesso.
Arquivo Data_CSV/poco_910.csv criado com sucesso.
Arquivo Data_CSV/poco_907.csv criado com sucesso.


In [11]:
pd.set_option('display.max_rows', None)
dlis_df_dict['907']

,TDEP,GR,BS,CAL,DRHO,PE,RESD,DTC,RHOB,NPHI,DCAL
3083,24.999983,None,None,None,None,None,None,None,None,None,NaN
3082,25.099983,94.170082,None,None,None,None,None,None,None,None,NaN
3081,25.199983,92.950974,None,None,None,None,None,None,None,None,NaN
3080,25.299983,88.086143,None,None,None,None,None,None,None,None,NaN
3079,25.399983,79.393135,None,None,None,None,None,None,None,None,NaN
3078,25.499983,76.099052,None,None,None,None,None,None,None,None,NaN
3077,25.599983,71.650536,None,None,None,None,None,None,None,None,NaN
3076,25.699983,65.908279,None,None,None,None,None,None,None,None,NaN
3075,25.799983,59.388111,None,None,None,None,None,None,None,None,NaN
3074,25.899983,60.084743,None,None,None,None,None,None,None,None,NaN


In [15]:
dlis_df_dict['900']

,TDEP,GR,BSZ,NPHI,CALI,RHOB,DRHO,PE,HDRS,DTC,DCALI
0,374.899994,None,8.5,None,None,None,None,None,None,None,NaN
1,374.799994,None,8.5,None,None,None,None,None,None,None,NaN
2,374.699994,None,8.5,None,None,None,None,None,None,None,NaN
3,374.599994,None,8.5,None,None,None,None,None,None,None,NaN
4,374.499994,None,8.5,None,None,None,None,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3595,15.399989,55.795601,8.5,None,None,None,None,None,None,None,NaN
3596,15.299989,52.5182,8.5,None,None,None,None,None,None,None,NaN
3597,15.199989,48.504501,8.5,None,None,None,None,None,None,None,NaN
3598,15.099989,45.021198,8.5,None,None,None,None,None,None,None,NaN
